# Overview
This notebook explores practical applications of MetaDoE through a hyperparameter tuning case study.

In [3]:
include("../../src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, Constraints, PSO, Objectives, Designs, Models
using Base.Iterators
using LinearAlgebra
using NPZ
using Optim

# Optimize Design

In [10]:
N = 20
K = 10

# Lower
lower_A = -I(K)
lower_b = [70 -500 -100 -10 0 0 0 -8 -1 -8]

# Upper
upper_A = I(K)
upper_b = [-20 1000 500 200 5 1 0.7 64 4 64]

# Factor constraints 
factor_A = [0   0  -1  10   0   0   0   0   0   0;
            0   0   0   0   0   0   0  -1   0   1]
factor_b = [0 0]

# Combined
A = Array{Float64}(vcat(lower_A, upper_A))
b = vec(hcat(lower_b, upper_b))

model = Models.quadratic
experiment = Experiments.create(N, K, model)
experiment = Experiments.with_linear_constraints(experiment, A, b)

Main.MetaDoE.Experiments.Experiment(Dict("8" => 8, "4" => 4, "1" => 1, "5" => 5, "2" => 2, "6" => 6, "7" => 7, "10" => 10, "9" => 9, "3" => 3…), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([-1.0 0.0 … 0.0 0.0; 0.0 -1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [70.0, -500.0, -100.0, -10.0, 0.0, 0.0, 0.0, -8.0, -1.0, -8.0, -20.0, 1000.0, 500.0, 200.0, 5.0, 1.0, 0.7, 64.0, 4.0, 64.0]), Main.MetaDoE.Models.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}(2, 0, true, Any[], false, true), 20, 10)

In [20]:
runner_params = PSO.runner_params(200, 200, 1e-6)
pso_params = PSO.create_hyperparams(500)

context = PSO.create_context(
    experiment, 
    Objectives.D; 
    hyperparams = pso_params,
    runner_params = runner_params,
    enforcer_type = ConstraintEnforcement.Parametric
)

runner_state, history = PSO.optimize(context)

Iteration: 0 Best score: -54.63486237431742
Iteration: 1 Best score: -73.71508458916553
Iteration: 2 Best score: -81.7161255886186
Iteration: 3 Best score: -83.35446602976766
Iteration: 4 Best score: -83.35446602976766
Iteration: 5 Best score: -83.35446602976766
Iteration: 6 Best score: -83.35446602976766
Iteration: 7 Best score: -83.35446602976766
Iteration: 8 Best score: -83.35446602976766
Iteration: 9 Best score: -83.35446602976766
Iteration: 10 Best score: -83.35446602976766
Iteration: 11 Best score: -83.35446602976766
Iteration: 12 Best score: -83.35446602976766
Iteration: 13 Best score: -83.35446602976766
Iteration: 14 Best score: -83.35446602976766
Iteration: 15 Best score: -83.35446602976766
Iteration: 16 Best score: -83.35446602976766
Iteration: 17 Best score: -83.35446602976766
Iteration: 18 Best score: -83.35446602976766
Iteration: 19 Best score: -83.35446602976766
Iteration: 20 Best score: -83.35446602976766
Iteration: 21 Best score: -83.35446602976766
Iteration: 22 Best sc

(Main.MetaDoE.PSO.RunnerState(Main.MetaDoE.PSO.Swarm(Main.MetaDoE.PSO.ParticleState([-31.794763497281586 -34.14075441244998 … -48.89851980855218 -20.0; -23.228783126759748 -55.831332031872975 … -42.97138005803763 -20.0; … ; -29.04966151961824 -32.65826284692937 … -50.176473694295154 -22.194334177099293; -21.35341081973176 -46.292932727051536 … -31.93719225370961 -28.2260187027491;;; 500.0 500.0 … 500.0 842.8012531893766; 869.7161163532891 500.0 … 500.0 548.9322059894353; … ; 691.7735230528549 500.0 … 500.0 500.0; 632.1927890522776 758.4468131257922 … 520.317418175731 770.24411496118;;; 342.2537716776662 201.62764503940332 … 298.2133666596912 352.1669509680986; 373.32365682610987 443.86697337427603 … 200.9707985436607 119.62160750745038; … ; 206.83413998720806 491.9081009254843 … 183.12913496287013 144.09844244521764; 225.53102357712433 500.0 … 100.0 462.6520450758429;;; 106.49053672270236 198.07919373068216 … 76.67492894600277 79.38761441152894; 106.5007359114357 72.37156571231331 … 10

In [21]:
PSO.save_results(runner_state; location = "../data/hyperparameter_tuning_case_study_2.npy")

In [25]:
PSO.save_results(runner_state; location = "../data/hyperparameter_tuning_case_study.npy")

# Fit Model

In [ ]:
lo

In [4]:
losses = npzread("../data/case_study_responses.npy")
settings = npzread("../data/hyperparameter_tuning_case_study.npy")

20×10 Matrix{Float64}:
 -48.7798  928.972  347.025   83.6256  …  0.264041  28.2619  1.0      54.7028
 -52.2183  500.0    442.73   174.128      0.148606  31.8837  1.85068  34.4703
 -49.0535  884.097  214.882  109.111      0.412729  59.7165  2.95611  12.3311
 -42.5272  934.262  282.425   99.5662     0.7       46.1146  3.66169  42.3605
 -53.2845  733.772  496.434  113.008      0.569163   8.0     2.2477   31.904
 -56.1692  500.0    304.894  111.643   …  0.583208  20.4567  1.2736   58.6492
 -61.4335  940.003  224.893   45.7421     0.10067   34.1157  3.30545  13.1938
 -68.6232  956.626  210.357  153.38       0.470373  54.8141  1.0      41.7612
 -31.8032  500.0    235.056  103.227      0.228527  57.3648  3.41393  55.3039
 -31.9794  946.234  128.346   10.0        0.175335  47.3142  2.1844   19.9742
 -55.5969  950.109  500.0    131.088   …  0.626139  36.1953  2.73472  43.9318
 -63.0352  500.0    412.807   92.6671     0.484378  60.5386  2.99447  46.2412
 -41.5332  528.21   499.616   77.9229     

In [8]:
model = Models.quadratic
F = model(settings)
β = F \ losses

21-element Vector{Float64}:
  7.863710156998575
  0.5232611886101487
  0.005256326738197957
 -0.009535172124199513
  8.560548045075197e-6
  0.003784669423712553
 -1.096860819055528e-5
 -0.028333049326344693
  0.0001415929180390749
  1.302164265884832
 -0.5332593804608906
 -5.87220266208906
  2.052017501864995
  7.128734005046965
 -5.936695688697819
 -0.08459447961840699
  0.0002574318539912251
  8.48729352283256
 -1.6838023344198507
  0.07464218866646075
 -0.0002910416359778379

# Obtain Loss-Minimizing Hyperparameters

In [ ]:
function surrogate_loss(x)
    return model(x)' * β
end

K = 10

# Use Fminbox to enforce box constraints
result = optimize(
    surrogate_loss,                  # objective
    -lower_b, upper_b,               # bounds
    (-lower_b .+ upper_b) ./ 2,      # initial guess
    Fminbox(LBFGS())                 # box-constrained optimizer
)

x_opt = result.minimizer